In [2]:
import pandas as pd # type: ignore
from datetime import datetime # type: ignore

In [3]:
import glob

folder_path = 'LJP'

# Get a list of all CSV files in the folder
csv_files = glob.glob(folder_path + '/*.csv')

# Create an empty list to store the DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
	df = pd.read_csv(file)
	dfs.append(df)

# Concatenate all the DataFrames together
df_arso = pd.concat(dfs, ignore_index=True)
df_arso.head()

,station id,station name,valid,T [°C],količina padavin [mm],globalno sev. [W/m2]
0,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:00,-2.6,0.0,0.0
1,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:10,-2.6,NaN,NaN
2,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:20,-2.6,NaN,NaN
3,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:30,-3.4,0.0,0.0
4,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:40,-2.9,NaN,NaN


In [4]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_arso):
	# Filter rows based on column: ' valid'
	df_arso = df_arso[df_arso[' valid'].str.endswith("00", na=False)]
	# Drop columns: 'station id', ' station name'
	df_arso = df_arso.drop(columns=['station id', ' station name'])
	# Change column type to datetime64[ns] for column: ' valid'
	df_arso = df_arso.astype({' valid': 'datetime64[ns]'})
	# Rename column ' valid' to 'datum'
	df_arso = df_arso.rename(columns={' valid': 'datum'})
	return df_arso

df_arso_clean = clean_data(df_arso.copy())
df_arso_clean.head()

,datum,T [°C],količina padavin [mm],globalno sev. [W/m2]
0,2022-01-01 00:00:00,-2.6,0.0,0.0
6,2022-01-01 01:00:00,-3.2,0.0,0.0
12,2022-01-01 02:00:00,-3.9,0.0,0.0
18,2022-01-01 03:00:00,-2.8,0.0,0.0
24,2022-01-01 04:00:00,-3.1,0.0,0.0


In [5]:
df_eles = pd.read_csv('ELES.csv', sep=';')

"""
Cell generated by Data Wrangler.
"""
from datetime import datetime

def clean_data(df_eles):
    df_eles['datum'] = pd.to_datetime(df_eles['datum'], dayfirst=True)
    df_eles['ura'] = df_eles['ura'].str[1:3].astype(int) - 1
    df_eles['datum'] = df_eles['datum'] + pd.to_timedelta(df_eles['ura'], unit='h')
    df_eles.drop(columns=['ura', 'predvidena_proizvodnja', 'dejanska_proizvodnja', 'nek', 'termo', 'hidro', 'predviden_prevzem'], inplace=True)
    # Filter rows based on column: 'datum'
    # df_eles = df_eles[df_eles['datum'] >= datetime.strptime('2024-08-01T00:00:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')]
    return df_eles

df_eles_clean = clean_data(df_eles.copy())
df_eles_clean.head()

,datum,dejanski_prevzem
0,2022-01-01 00:00:00,1044.0
1,2022-01-01 01:00:00,1018.0
2,2022-01-01 02:00:00,975.0
3,2022-01-01 03:00:00,944.0
4,2022-01-01 04:00:00,936.0


In [6]:
df_pokritost = pd.read_csv('LJP_pokritost_2022-2024.csv')
df_pokritost['datum'] = pd.to_datetime(df_pokritost[' valid'])
df_pokritost = df_pokritost.drop(columns=['station id', ' station name', ' valid'])
df_pokritost.head()

,oblačnost [%],datum
0,37.0,2022-01-01
1,40.0,2022-01-02
2,80.0,2022-01-03
3,100.0,2022-01-04
4,100.0,2022-01-05


In [7]:
df_prazniki = pd.read_csv('prazniki.csv', header=None)
df_prazniki['datum'] = pd.to_datetime(df_prazniki[0])
df_prazniki = df_prazniki.drop(columns=[0])
df_prazniki = df_prazniki.rename(columns={1: 'holiday'})

# Generate a new DataFrame with all dates between 2022-01-01 and 2024-10-23
df_all_dates = pd.DataFrame({'datum': pd.date_range('2022-01-01', '2024-10-23')})

# Merge the new DataFrame with df_prazniki, filling in missing values with 0
df_prazniki = pd.merge(df_all_dates, df_prazniki, on='datum', how='left').fillna(0)

df_prazniki.head()

,datum,holiday
0,2022-01-01,1.0
1,2022-01-02,1.0
2,2022-01-03,0.0
3,2022-01-04,0.0
4,2022-01-05,0.0


In [9]:
df1 = pd.merge(df_eles_clean, df_arso_clean, on='datum')
df2 = pd.merge(df1, df_pokritost, on='datum', how='outer')
df = pd.merge(df2, df_prazniki, on='datum', how='outer')

"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Drop duplicate rows in column: 'datum'
    df = df.drop_duplicates(subset=['datum'])
    # Renames
    df = df.rename(columns={'datum': 'date', 
                            'dejanski_prevzem': 'consumption',
                            'T [°C]': 'temperature',
                            'količina padavin [mm]': 'precipitation',
                            'globalno sev. [W/m2]': 'irradiation',
                            'oblačnost [%]': 'cloudiness'
                        })
    df.ffill(inplace=True)
    df = df[(df['date'].dt.day != 29) | (df['date'].dt.month != 2)]
    df = df.drop(df.index[0])
    return df

df = clean_data(df.copy())
df.head()

,date,consumption,temperature,precipitation,irradiation,cloudiness,holiday
1,2022-01-01 01:00:00,1018.0,-3.2,0.0,0.0,37.0,1.0
2,2022-01-01 02:00:00,975.0,-3.9,0.0,0.0,37.0,1.0
3,2022-01-01 03:00:00,944.0,-2.8,0.0,0.0,37.0,1.0
4,2022-01-01 04:00:00,936.0,-3.1,0.0,0.0,37.0,1.0
5,2022-01-01 05:00:00,952.0,-4.1,0.0,0.0,37.0,1.0


In [11]:
df.to_csv('dataset.csv', index=False)